# Get CBA Policy Documents
CBA make donwloading the policy difficult by
1. Having a SSO based login that makes request-http automation difficult
2. Using ASPX to dynamically load content.

over come this using by loggign in with the browser, then ctting and pasting the auth cookies into this notebook.

### Setup

In [2]:
import os
import re
import requests
from urllib.parse import urlparse, urljoin
import shutil

from loguru import logger

from IPython.display import HTML
from bs4 import BeautifulSoup

### Scraper Code

Notes
1. Only searchs for links in div#wrapper
2. Special code to deal with ASPX/DynamicContent
3. Limited to

In [3]:
def fetch_content(url:str, cookie:str):
    ''' Get the contents of a URL using a logged in cookie'''
    session = requests.Session()

    # Send the request with the cookies
    try:
      response = session.get(url, cookies={'Cookie': cookie})
    
      if not response:
        logger.warning(f"No response for {url}")
        return ""
      if response.status_code != 200:
        logger.warning(f"Response {response.status} for {url}")
        return ""
    
    except Exception as e:
      logger.error(f"Error {repr(e)} for {url}")
      return ""
    
    return response.text

def save_url(url:str, download_path:str, cookie:str) -> str:
  '''Save the url to the download path and return the contents'''
  html_content = fetch_content(url,cookie)

  full_path =  download_path + urlparse(url).path
  directory = os.path.dirname(full_path)
  
  if not os.path.exists(directory):
    os.makedirs(directory) 

  logger.info(f"Downloading {url}")
  with open(full_path, 'w') as f:
      f.write(html_content)

  return html_content

def aspx_link_to_html_link(aspx_url, cookie) -> list[str]:
  '''Extract the html links from the aspx file'''
  
  aspx_content = fetch_content(aspx_url, cookie)

  # Links in dynamic content function
  pattern = r'LoadDynamicContent\(([^,)]+),([^,)]+),([^,)]+)\)'
  matches = re.findall(pattern, aspx_content)
  if len(matches) >0 :
    # return  [urljoin(arg1,arg2) for arg1, arg2, _ in matches]
    return  [(arg1 + arg2).replace('"', '').replace(' ', '') for arg1, arg2, _ in matches]
  
  # or links in an input tag
  pattern = r'<input[^>]*value="([^"]*\.html)"[^>]*>'
  matches = re.findall(pattern, aspx_content, re.DOTALL)
  if matches:
    return matches
  
  logger.warning(f"{aspx_url} no html URL")
  return []
  
def fetch_html_links(html_page, class_names):
    '''Find all the links in the page'''
    soup = BeautifulSoup(html_page)

    links = []
    for class_name in class_names:
        for link in soup.select('div#wrapper a'):
                    if href := link.get('href'):
                        if href.startswith("/Net/Documentum/"):
                            href = domain+href
                        if not href.endswith('.aspx'):
                            print("Not an ASPX", href)
                        links.append(href) 
    return links

cache = set()
domain = r"https://www.commbroker.com.au"
def scrape(url:str, cookie, download_path:str, depth:int, starting_page:bool, search_only) -> None:
    # Stop if when we get to the bottom
    if depth and depth <= 0:
        logger.info(f"Bottomed out at {url}")
        return 
    
    path = urlparse(url).path
    extension = os.path.splitext(path)[-1].lower()
    
    # skip already processed urls   
    if path in cache:
        return
    
    cache.add(path)

    if not url.startswith(domain):
        logger.info(f"Skipping domain {url}")
        return

    # Skip unwanted videos etc
    if extension not in [".html", ".aspx"]:
        logger.info(f"Skipping unwanted extention  {url}")
        return
    
    if "Logon.aspx" in path:
        return

    # If aspx find the base html then extract the links and recurse without saving content
    if extension == ".aspx":
        html_urls = aspx_link_to_html_link(url, cookie)
        for html_url in html_urls:
            scrape(html_url, cookie, download_path, depth, starting_page, search_only)
        return
    
    # download all resources except starting page
    if starting_page:
        html_content = fetch_content(url,cookie)
    else:
        if search_only:
            logger.info(f"Saving {url}")           
            html_content = fetch_content(url,cookie)
        else:
            html_content = save_url(url, download_path, cookie)

    # if we have not yet exceeded the depth find all the links in wrapper and recurse
    if extension == ".html" and depth and depth > 0:
        soup = BeautifulSoup(html_content)
        for link in soup.select('div#wrapper a'):
            if href := link.get('href'):
                if not urlparse(href).netloc:
                    href = urljoin(domain, href)
                
                scrape(href, cookie, download_path, depth-1, starting_page=False, search_only=search_only)
    return

# Download Comm Broker

In [4]:
directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/CBA/CommBroker/"
cookie="ASP.NET_SessionId=3ddhlzaz01va0301t10thyww; cbid=XCE0lZxtdgKJPk3uhc6IXUfYUl7TOwFF-qsh2Ba69AUQivik3Tswg7QYSvcaNp0G8RulxEhT5owxBuSrWO_a7x_TciMcRxNHazWDObRBv-l9FUPuK3jFaCSJnJItrJ6rUl9-57rmOpZg9Tpkt23lFecCUbIjzGiWhKCaelotWB9xEdA_YSnxvvjmQPKYlWXJsoOU2tljiTaZ0x-pUbr4DCg58RPTu73ATq8wbOql3TdLZ5Poa6BZPKyjDV41RFuWSiggHofZiB6wD-pOpPRnji9QH8Qag-vF23s2Ek6vmPWQN0oP7m65y-sHE2PFl3qQ5EOQfTZ8W0aWQsA_ZONMpoFJrutj8yFilIBFfoOuBIiCY56eU89ZOhhFKnoubQaF0xNxxBh1d9RcyeLZ-L389Fy9iJ4wnFLdlMCKUd2G8wqKhq2Mf25Vk2HWYOJyWTnAzu--p7pm15-hQkaZHsgNAokbTi7bdpi7FAc9UAZKPNUylcNO7uKiXkIePUyxJUY33qAwbOHNMLqp0VSGxzM4dH48fQr4595366JzlEWh0QVuPUwQfhWLxhiUd8_ded01M8eb0GzhUaf7n1GSyfG5bAizbULMi4nzzB1gdvyoeybLP4cmrXfuwTq1ssSje3GDkPdUSOeC2G2l2tWCFoI9h4Z1rvd6uQ8x4mSYci7bVY1PSxNX1KK0whEa-Rg_qeNE-dhe0OsS5uBFUDTMKw8ecLUUvRPwFtqmtW5V-IC0eKrEUW5jDWqjrmjN2qII3QeoVV0et9Rr2bOUq-W8-xXOMCvv1eB2FLRU5K5UoWbZNnHdesVtAFxiJyN0UyGLZphJFDi_OxINu0pu7TdqMIvCoW8EecMpttMj9MmDBLY1gcN3fEO5tNdS6r2xKzCz33YXB-soDH4GOlHw8rrjnapcBsNmdGvqdb2FPMNTHXdlQzz3AUYuJShL1vnKW5rgElpP423IucFyXFH0RMBSO_q4rrvezLeIAlIP3H9szXSQ7TMBo1kpwZDPGvfwD3MkYeyBc1MeV6Vr01jpzQX__rCL0kTamQJKGSdUzdLEjFLgv_uJiy48pSLyY1qxu_7bpgjS-WGrpOLDvaqt2wRxEuv87zAqPLxfUL8f0eZtAYm0LOxL7EG-ktllpJt4FE1VixfHfdoYmMubauvrtGhReTyF0lqT4N6SyV2mONhrSIuZ4zAtR6yRisMhhyhNN7rhgmI4YQnEIbgHavnsUwKKlX9yGmA1ALuSjVtUcFgzORipP_gcR3MBuyHYA4n1zCfkACTBJvorJh_sYhCoJ1DU9Z23v8tRvNAO0iFwmZdzP1qahnGAAY4n_eTCFgBajNADO5Ly_9sw-2dI4caeejtf4Mj67DLmhxV36PnOJZjEj1zRD8Mfl9qEMuBcZbLFyDrOovWUk0qry8mBjam6m3_OHCjEBk8rQNcfOtE9_qmRmarfz8Cspt3gIr5WK9_fcOGtwXrgzFrUUrn9rol3ILzEwxt4YpcgfnxT5aS57yTutWdLVttsnR6z_pDMqFAlcL9zkZSjICqBqg2rdZ9IN_UYB3rdwosuQcu8YhIEx_V1m7Hqj58hjQjrLNDx1axW8cSDVh8ryeCdPLSXRbvuLNmGYBprdgjZeY22nDYDRVy4DzqZmTLrTAmNT2rsv8AaRZoFOTw7fA9hD2Ru7O6Oc6zMi64Q6k5GOPMFY72wduX9GzdcTJygmETv_oxKcrwPSRsYTSAaKxvwT5vdEUuqsZSCRHdwYVNdrqqu4M6A5khOhMUaAtZ1mc82CbspmhEaLaQyO29OltSdidHzdXdQVqK-JLcZZJV6LQIo356524lLFRgSpVYC-913PaqFMGx39Ib7oZdhDcY-VKJJg7NJQxBu_HV4yrsbFCfrw7a8oR_kdPVkNUKJGBLFP1yafE2l_Ww8RnnSHFqVNUps88xjdZcGy3kvq4asrWCC8JPiSQOGD6NaeEVzGa1ZavaEAmlXeWjQQDqcnNJFbLepBWhpDF11_J-FmcS1dIwf-O0RwbsQwo0R7bncN2xdf8rffcFGXhDogSAVVkY3jWhd1sO83gCJy0WVtWXPKJm7w3qHGIwKnnYmLxCTJqu6tkNnli57QnTv8IR0rKQSJ_0CNdNY96niHPQfkkodQ63FXwP5xIsNnA5psHfztG0_lO0oBYc2PiI7fiKxj40ZEiulSXKzUFojL-6BKcmiGvIFemrAiV6zZM-CvbXBwR2_-_7qbeNipnjFOoQYRAVmV9ThK4LZyPGpNnjM6r_Zlc6xuWo-DgRTshZfD0FvIqBRYvcfiADXD5-vjwAPN6XaEX7YYHdbyrnbKwd_oKJ5M1t_UTirphV6Ie96ShlhviErBrOtbwYWiTK4m8Xpf-0fP__Gl6Yb1vtvB_Xlm_vDZWg3FHxnkmWpUBnvSQlHRJ6tZ2HaWDVOFEzyv34Taj8Jh_na-BdFzaGNDaJvFXFIQyK5_z-LUVy1bIq5cFUeJzqIOKptMTDyrXSYKdCguOzkoN5k4xygWy5F3X6KH5bg8naTflqenJv5Z7JirxLEt1daO66dmvuy5bUXN3byI25fMsB8UmfUU-YhuozDoLhy0YqfcfjPdrOqXt56Yx1tlVlTXxQYRZ1Jcmtou8S6F1M06jS6pmXTPePlbs_aMWkuBVuWcmss85-u6DNx0MQg93RQhPUp8TvILJit5H-iKQp1BZrsidG5bSN-Lu7X6SRbHj9UXuPitJ8Yn4ujsIIP1tO85LYnbuK2ytrHl5-0eEAILenRqur8v69NwM9qPf-o3Tm6fdc1Da15rqKyMhErwodXEBEgthX-sBLYPVNkZbtoOO5T_TdUDqO6azCy_SjtvVuaE6sGsSmUYlhT3TZBM2N1Ve11QpVABd0IruDaF60sOdG6Xc9NJQcnnrJsPMbdC8XuBYezsUOSUVKpICyNN66Dfnmi5_olo7vCB3UQedOnSvBQuBps34D2dzyNl4Ad3nOudw9MO9WBfiQeuD1gpZHZtU19rxftvGqRXU8bVZxUJEiTXOx6xlPShsJkMZa8dwpOCde2ZkD7AWIKLcuW6W5xY5v9ZJyWv2e2BbCYjt-7R8ZDVeK1CjuSZrl98uomdvlvunEOQ1qBjgEY_AwsqeNCI7lBSY34J73PTbi7vm7UGmavoeMXiRab05dPm3rw6NCxsVlQUWE1tzXHHGljhob6LO3TGWI3YS5XBRwC1TIPdWqicMAKDM_6NpTZX2cgV1_Zehy0FgPERZ3Lwx7ljzSBPrMYwLXSbPz2le_CmIoKFy6Ijasn08vLikcekqn_KZVGwd5kL4sZho9DKlY0-Zday1dzIJAZSEJf-w6upSXR9a6I0f0mSinlefp6VRoq0MPaSq36AX9rOQ5b9lO4sSjsSGHIVJ7-w1FEt4fqAAogTt5_1OIY6HhonINsF4OlZXKxmziLJbLDfyn9kQNCRwzRZ-XpmZpZnvCwrjBjctBCzg; bm_mi=093E03C6C35EA4C4317588061B78D430~YAAQXgNiaP7+TymUAQAAfF74KRpGviOHKpPSML2xlHdNDqE73/7vD55aZkpdnPd3QXWAkSd3dEQKNBN7E9JAHJMsqx/CQ9BImzq10r3C+0IQ3n3EdMZ68rk8RTue7lkokrlySThRUaK28+YNy4JFovVcAso+hpWsuJwv8j+Sr7B/N7WEX9CPsJCpZsGDWMmKBQWH71uYP4bVi1hCEHJyHo104bibKDh+vZsyCiMEq1RZpmdYz6dgs7FiLSpkoCpnyNx6Iny8mfvWnLh50qnex1AJFqc831jUjzmiU9vhjBvkxYiR8oe5ia1mxRqCp2DkyNEmezPiaHc4Gl39SMGupbrkgu0wAmp1WpUQ0LWpUNeHDpVCVAzX8S68lES75Q==~1; bm_sz=BDE6D8676CA59247EFDB575D081DA6E9~YAAQXgNiaAD/TymUAQAAfF74KRriVbEneNyZ5TOP95XjdY/sG2XlZPxAt8GXIhNQmajpPA3S5an3u/U+4jB65fJzr6S5JcpOD6p4Py8NNROzkhvNsky4hsz39XH1QDZWIWhgKaZulNIQ7gIdSS7brfz93ekuc8bUGwCJzUWeKDXcyhRCjjEi+7+oRiJAaKcGKw59c3IgSu4pr5XZAPoGp0lux7VZ+l8dORyzvB6dgIeV2Y5trG5PoZKLZy6pnANjoVjqJEnlBakD8awFMdhBWUplrX87O5Cj1Zl6vsGeeaZF/k9kdc/B5SchG+EVfN3a7cc1xbCUVZtn91fheB/Izpv3gdBhteK9c5fzdfCOYKJcGJIxKp8isHCdSobz3oDc7QPM2ujMduSLCGUMEvQMZ6I+iHb9r5aUuOnUQYuatg==~3617841~3290177; ak_bmsc=2D09173FE20992C8DBA1592725E508AF~000000000000000000000000000000~YAAQXgNiaHv/TymUAQAA1WL4KRpfJ+TH8HzUpKNiHHSsq3meZNqnqr/AOXK4Dqnhyxa6DcAZGY5XsKqlgltgcz/6yX94Vbc4cnhF0sSsXLEAnnag4cv6D1b42rfSUfpgCeWrabnmx/nDUbe6VeXtECIZKvLzJVHQ/AFQT9gyqvgMnBtvsHTfgn6t2KMdTlC2LP9YLvDyeGE2j5uHHkUNOoUcQgHWE58di9LMz4Xzk01DeFjZHCDJn3HkgaHY/rFfYct5rbap8qGmnNQGze5k+emKs9tKknPUeQ3ABo81A4f8dKDa76tUk7n7HluUaIAPDyCGmmilzWi2BQO0jODf/YKGXT30MdCaiFEhSIBnS/MCrHC6MVCddlNet86eIpOHP/z2l8KEL1dH+M5dX+in3FAXbleoamLLeq11vW0CC2Fc+tyUf3kVP0DqhTf97CUOZxuf7BgIahDq8LLZqjmPhlsbzdcb/Lr4zJS94XrGaDsBa+SpgG7N0omX41532IZrrDPmlum4qMizBFmI7IkM2faVXaGg+KA=; cbpopup=50; s_cc=true; htsm=exp=1735872743~acl=/*~hmac=4d455dd8ba949bfa065b6a871e4d06b29711cc05c04feee69a8278db6890b9e8; bm_sv=9FE7D3A4A85B5D0B2A84976018C4EDE0~YAAQXgNiaFNWUSmUAQAA3UQDKho5kLS93zkTdh/eIdDiUeSZ1xSPbKWvT1ANOexNf5FLqu6wwlysnEpY/rZNzJc2/Mgm2mcR6u6AqfsefDkDyj4gPEbo+gSn+yPtCoik15pgGR/oZIioPV1VAFhDnaO+HKeT/l59iD6uF97PN1/uoc6K6+Q0cqVOCRxnQ0uXRbjgjg7ld6Y4n8FJ5y/vGufA/Z59GAeaUKGMrYX6txPhqjtYyv+iAWmw3+HSJCpCRbDPh5+DVw==~1; _abck=64879BC9507C9570BF71DAA593F45962~-1~YAAQXgNiaBhYUSmUAQAA71UDKg3RGlP2TqGd/VHZx2THCdYXThiAiaHQ3DBQpmR3nMI8W1nIXeX2+PELfXPejjSay/9i3rrF8iX//hYB0NRH0IxRxAC9FQWMehg3vCMGWvTto9E78vQicpUGUn7kJhseG3JMu14RoEk0AcZ/RPHoLgWrz/2dslnvHa1iMCP6vjyYEsfgPxnqOXuadlLe9a9yNsXkQZ5Iu1ULddo3hD8ZWa/s+ArUIFng/0kFf2SYFTS4dgJdETwfHlgLzr7szTI8ROlWbNZYyc4/XdcwMuCFuxN6PQdt/bGMJMRYa0VfQ6KssTFpZiuyqEZESjbZFn7Ja+8FVs61yYwtnepzshpQSWXPlIkCMw8An8dmei+JubXvOGGz8CbtjGKC8VGGsyahm0rMMiEGW0g7+RgP3jpRM9z8Q2svfKrQxPQpHp0LXo6XqerHliWBcu76Pn0hhOsHPEKvrUMviwo9aujGyMADhD/o3/PdzeZXIXQ5asu1Fq2UPo9M45mNBLvnc3w=~-1~-1~-1; s_nr=1735871648417; s_sq=cfsadviser-cb-prd%3D%2526pid%253Dcb%25253Apopup%25253Aclosebutton%2526pidt%253D1%2526oid%253Dhttps%25253A//www.commbroker.com.au/Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx%2526ot%253DA"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

#Need depth 3 to get to 
cache=set()
scrape("https://www.commbroker.com.au/Net/Documentum/processes/default.aspx" ,    cookie, depth=3, download_path=directory_path, starting_page=True, search_only=False)
scrape("https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx", cookie, depth=3, download_path=directory_path, starting_page=True, search_only=False)

2025-01-03 10:37:23.780 | WARNING  | __main__:aspx_link_to_html_link:56 - https://www.commbroker.com.au/Net/Documentum/default1.aspx no html URL
2025-01-03 10:37:24.274 | INFO     | __main__:save_url:32 - Downloading https://www.commbroker.com.au/content/commbroker/en/private/latest-news/live-updates.html
2025-01-03 10:37:24.773 | WARNING  | __main__:aspx_link_to_html_link:56 - https://www.commbroker.com.au/Net/Documentum/default.aspx no html URL
2025-01-03 10:37:24.778 | INFO     | __main__:scrape:92 - Skipping domain https://www.commbank.com.au/latest/support-for-home-loan-customers.html?ei=pers_HL
2025-01-03 10:37:24.781 | INFO     | __main__:scrape:92 - Skipping domain https://www.commbank.com.au/latest/coronavirus.html
2025-01-03 10:37:24.786 | INFO     | __main__:scrape:92 - Skipping domain https://www.commbank.com.au/latest/home-loan-direct-debit-repayments.html
2025-01-03 10:37:24.788 | INFO     | __main__:scrape:97 - Skipping unwanted extention  https://www.commbroker.com.au/c